Urban Data Science & Smart Cities <br>
URSP688Y Spring 2025<br>
Instructor: Chester Harvey <br>
Urban Studies & Planning <br>
National Center for Smart Growth <br>
University of Maryland

# Demo 7 - Geospatial Data

- Coordinate systems
- Points from XY
- Loading shapefiles and geojsons
- Proximity analysis
    - Measuring distance
    - Buffering
- Overlay analysis
- Spatial joining

## Geospatial data fundamentals
- Basic geometry types:
    - Points
    - Linestrings
    - Polygons

<img alt="points, lines, and polygons" width=500 src="https://datacarpentry.org/organization-geospatial/fig/dc-spatial-vector/pnt_line_poly.png">

- Spatial analysis (with vector data) is essentially just Euclidean geometry (remember the Pythagorean theorem?)

<img alt="pythagorean theorem" width=500 src="https://www.katesmathlessons.com/uploads/1/6/1/0/1610286/published/how-to-use-the-pythagorean-theorem-to-find-distance-between-points-on-coordinate-plane-2.png?1595954050">

## Geospatial data are everywhere

DC affordable housing: https://opendata.dc.gov/datasets/DCGIS::affordable-housing/about

In [ ]:
# Install Geopandas

# ! conda install -y geopandas

In [ ]:
# Import dependencies
import pandas as pd
import geopandas as gpd

## Make points from CSV

In [ ]:
# ! Load CSV

In [ ]:
# ! Use gpd.points_from_xy to make geometries

In [ ]:
# ! Use geometries to make a geodataframe with CRS

In [ ]:
# ! Plot geodataframe

## Measuring proximity

### How far is each affordable housing project from Metro Center?
- Map projections

In [ ]:
# Let's start by reloading the data with more descriptive variable names so we don't get confused
projects_df = pd.read_csv('Affordable_Housing.csv')
projects_gdf = gpd.GeoDataFrame(projects_df, geometry=gpd.points_from_xy(projects_df['LONGITUDE'], projects_df['LATITUDE']), crs=4326)

metro_center_lon = -77.0312124
metro_center_lat = 38.898202
metro_center_gdf = gpd.GeoDataFrame(geometry=gpd.points_from_xy([metro_center_lon], [metro_center_lat]), crs=4326)

In [ ]:
# What's wrong here?
projects_gdf.distance(metro_center_gdf.geometry.iloc[0])

## UTM Zones

[Universal Transverse Mercator](https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system) Zones

The UTM system divides the globe into 120 zones, 60 each in the northern and southern hemispheres. It uses Transverse Mercator projections to minimize shape and distance distortion for localized measurements within each zone.

<img alt="utm slices" width=500 src="https://gisgeography.com/wp-content/uploads/2016/05/UTM-Zones-Globe-2.png">

<img alt="utm standard line" width=500 src="https://gisgeography.com/wp-content/uploads/2016/05/Universe-Transverse-Mercator-Cylinder.png">

1. [What UTM zone am I in?](https://mangomap.com/robertyoung/maps/69585/what-utm-zone-am-i-in-#)
2. [What is the EPSG code for that zone?](https://www.google.com/search?q=epsg+code+for+UTM18N&rlz=1C5GCCM_en&oq=epsg+code+for+UTM18N&gs_lcrp=EgZjaHJvbWUyBggAEEUYOTIHCAEQIRigATIHCAIQIRigATIHCAMQIRigATIHCAQQIRigATIHCAUQIRigATIHCAYQIRiPAjIHCAcQIRiPAtIBCDcxMDJqMGo3qAIAsAIA&sourceid=chrome&ie=UTF-8)

In [ ]:
# Reproject into the local UTM zone
utm18n = # ! Look up EPSG code for UTM zone for Washington DC

# ! Project both projects and metro center geodataframes into utm

# ! Calculate distance between each project and metro center

In [ ]:
# ! Get projects that are within a mile of metro center

In [ ]:
# ! Map projects with .explore()

### Which projects are within 1 mile of Metro Center or Columbia Heights?

#### Another approach: Buffer and spatial join

In [ ]:
# Define multiple locations
metro_locations = {
    'Metro Center': {
        'lon': -77.0312124, 
        'lat': 38.898202
    },
    'Columbia Heights': {
        'lon': -77.0350154, 
        'lat': 38.92890
    },
}

# Convert to a geodataframe
metro_locations_df = pd.DataFrame.from_dict(metro_locations, orient='index')
metro_locations_points = gpd.points_from_xy(metro_locations_df['lon'], metro_locations_df['lat'])
metro_locations_gdf = gpd.GeoDataFrame(metro_locations_df, geometry=metro_locations_points, crs=4326)
metro_locations_gdf = metro_locations_gdf.to_crs(utm18n)

In [ ]:
# Buffer metro locations
metro_locations_buffer = metro_locations_gdf.buffer(threshold_dist)

In [ ]:
metro_locations_buffer.explore(tiles='CartoDB positron')

In [ ]:
metro_locations_buffer_gdf = gpd.GeoDataFrame(geometry=metro_locations_buffer)
metro_locations_buffer_gdf

In [ ]:
projects_near_metro = gpd.sjoin(projects_gdf, metro_locations_buffer_gdf)

In [ ]:
projects_near_metro.head()

In [ ]:
projects_near_metro = projects_near_metro.rename(columns={'index_right':'near_metro'})

In [ ]:
projects_near_metro['near_metro'].value_counts()

In [ ]:
projects_near_metro.explore(column='near_metro', tiles='CartoDB positron')

## Overlay

### How many projects are in each census tract?

- [Population by Tract from Census Reporter](https://censusreporter.org/data/table/?table=B01003&geo_ids=16000US1150000,140|16000US1150000&primary_geo_id=16000US1150000)

In [ ]:
# Load tract polygons from geojson
tracts_gdf = gpd.read_file('acs2023_5yr_B01003_14000US11001007409.geojson')
# Project into local UTM
tracts_gdf = tracts_gdf.to_crs(utm18n)
# Only keep records that are tracts (there's pesky one for all of Washington DC)
tracts_gdf = tracts_gdf[tracts_gdf.geoid.str.len() == 18]

In [ ]:
tracts_gdf.plot()

In [ ]:
# Ensure projects and tracts are in the same projection (CRS: coordinate reference system)
assert projects_gdf.crs == tracts_gdf.crs

In [ ]:
# Spatially join projects to tracts
tracts_with_projects_gdf = gpd.sjoin(tracts_gdf, projects_gdf, how='left')

In [ ]:
tracts_with_projects_gdf.head()

In [ ]:
# Count unique projects in each tract
project_counts = tracts_with_projects_gdf.groupby('geoid').index_right.nunique()

project_counts

In [ ]:
# Join counts onto tracts
tracts_with_counts_gdf = tracts_gdf.merge(project_counts, left_on='geoid', right_index=True)

In [ ]:
tracts_with_counts_gdf.head()

In [ ]:
# Cleanup columns
columns = {
    'geoid':'geoid',
    'B01003001':'population',
    'index_right':'projects',
    'geometry':'geometry'
}
tracts_with_counts_gdf = tracts_with_counts_gdf[columns.keys()].rename(columns=columns)

In [ ]:
tracts_with_counts_gdf.head()

In [ ]:
tracts_with_counts_gdf.plot(column='projects')

In [ ]:
# Calculate projects per population
tracts_with_counts_gdf['projects_per_population'] = tracts_with_counts_gdf['projects'] / tracts_with_counts_gdf['population']

In [ ]:
tracts_with_counts_gdf.plot(column='projects_per_population')

### How many units are in each census tract?

In [ ]:
# Use a the general 'agg' method to aggregate multiple columns at once by different methods
project_stats = tracts_with_projects_gdf.groupby('geoid').agg({
    'index_right': 'nunique',
    'TOTAL_AFFORDABLE_UNITS': 'sum',
})

tracts_with_counts_gdf = tracts_gdf.merge(project_stats, left_on='geoid', right_index=True)

columns = {
    'geoid':'geoid',
    'B01003001':'population',
    'index_right':'projects',
    'TOTAL_AFFORDABLE_UNITS': 'units',
    'geometry':'geometry',
}
tracts_with_counts_gdf = tracts_with_counts_gdf[columns.keys()].rename(columns=columns)

In [ ]:
tracts_with_counts_gdf.plot(column='units')